# 🔎 Step 1.1 — Import Required Libraries
### Why this step?

Before scraping and analyzing EV data, we need to import essential Python libraries:

- **NumPy & Pandas** → For data handling and analysis  
- **Seaborn & Matplotlib** → For visualization  
- **re (Regex)** → For text pattern cleaning  
- **requests & BeautifulSoup** → For web scraping (sending HTTP requests and parsing HTML)  
- **warnings** → To suppress unnecessary warning messages  

This setup ensures we have all necessary tools for scraping, cleaning, and analyzing EV data efficiently.


In [1]:
# Basic libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import requests
import time
import warnings
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore")


# 🧩 Step 1.2 — Initialize Data Containers for Scraping
### Why this step?
Before starting the scraping process, we prepare empty Python lists to store data for each EV attribute — such as name, range, efficiency, weight, towing, and so on.  
Each list will hold one column’s worth of information, which we’ll later combine into a single DataFrame.  

✅ This structure ensures our scraped data remains organized and ready for tabular conversion.


In [2]:
# Initialize empty lists for each EV attribute
car_name = []
availability = []
range_km = []
efficiency = []
weight = []
acceleration = []
battery = []
fastcharge = []
towing = []
cargo = []
price = []
year = []
body_type = []
page_num = []


# 🧩 Step 1.3 — Scrape Car Data from the EV Database (Main Loop)
### Purpose:
This step iterates through 10 pages of the EV Database website, extracts details for each electric vehicle (e.g., name, range, efficiency, battery, towing capacity, etc.), and stores them in lists.

### What Happens:
1. A loop runs from pages 1–10.
2. For each page:
   - The script fetches HTML content from the EV Database main page.
   - BeautifulSoup parses the page.
   - All `<div>` elements with class `list-item` (representing cars) are found.
3. From each car block, key information is extracted (if available) — otherwise `NaN` is appended.
4. Results are stored in lists (`car_name`, `availability`, `range_km`, etc.) for later conversion into a DataFrame.
5. The code prints the scraping duration for each page and total time at the end.

### Why Important:
This section collects the raw car data that forms the foundation for further cleaning and analysis (EDA).


In [3]:
total_start = time.time()

for i in range(1, 11):
    start_time = time.time()
    print(f"Scraping Page {i}...")

    # The base site (EV database main listing)
    url = "https://ev-database.org"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    cars = soup.find_all("div", class_="list-item")

    for car in cars:
        # Extract car attributes safely
        name = car.find("a", class_="title")
        avail = car.find("div", class_="availability")
        rng = car.find("span", class_="erange_real")
        eff = car.find("span", class_="efficiency")
        wgt = car.find("span", class_="weight_p")
        acc = car.find("span", class_="acceleration_p")
        batt = car.find("span", class_="battery_p")
        fc = car.find("span", class_="fastcharge_speed_print")
        tow = car.find("span", class_="towweight_p")
        carg = car.find("span", class_="cargo")
        prc = car.find("span", class_="pricefilter")
        yr = car.find("span", class_="year_from")
        shape = car.find("span", class_="shape-suv")

        # Append values or NaN if not found
        car_name.append(name.text.strip() if name else np.nan)
        availability.append(avail.text.strip() if avail else np.nan)
        range_km.append(rng.text.strip() if rng else np.nan)
        efficiency.append(eff.text.strip() if eff else np.nan)
        weight.append(wgt.text.strip() if wgt else np.nan)
        acceleration.append(acc.text.strip() if acc else np.nan)
        battery.append(batt.text.strip() if batt else np.nan)
        fastcharge.append(fc.text.strip() if fc else np.nan)
        towing.append(tow.text.strip() if tow else np.nan)
        cargo.append(carg.text.strip() if carg else np.nan)
        price.append(prc.text.strip() if prc else np.nan)
        year.append(yr.text.strip() if yr else np.nan)
        body_type.append("SUV" if shape else "Other")

        page_num.append(i)

    print(f"✅ Page {i} scraped in {time.time()-start_time:.2f} seconds")

print("✅ Total scraping time (seconds):", round(time.time()-total_start, 2))


Scraping Page 1...
✅ Page 1 scraped in 23.83 seconds
Scraping Page 2...
✅ Page 2 scraped in 22.41 seconds
Scraping Page 3...
✅ Page 3 scraped in 26.76 seconds
Scraping Page 4...
✅ Page 4 scraped in 24.80 seconds
Scraping Page 5...
✅ Page 5 scraped in 20.74 seconds
Scraping Page 6...
✅ Page 6 scraped in 21.49 seconds
Scraping Page 7...
✅ Page 7 scraped in 20.72 seconds
Scraping Page 8...
✅ Page 8 scraped in 21.36 seconds
Scraping Page 9...
✅ Page 9 scraped in 22.77 seconds
Scraping Page 10...
✅ Page 10 scraped in 22.11 seconds
✅ Total scraping time (seconds): 227.0


##  🧩 Step 1.4 — Create a Data Frame


In [4]:
df = pd.DataFrame({
    "Car Name": car_name,
    "Availability": availability,
    "Range (km)": range_km,
    "Efficiency (Wh/km)": efficiency,
    "Weight (kg)": weight,
    "Acceleration (0-100s)": acceleration,
    "Battery (kWh)": battery,
    "Fastcharge (kW)": fastcharge,
    "Towing (kg)": towing,
    "Cargo (L)": cargo,
    "Price (€)": price,
    "Year": year,
    "Body Type": body_type,
    "Page": page_num
})

print(f"✅ Total records collected: {len(df)}")
df


✅ Total records collected: 11450


Car Name  \
0              BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1           MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2           Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+   
3                         BMW iX xDrive40BMW iX xDrive40   
4      Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...   
...                                                  ...   
11445  Opel Vivaro-e Combi M 50 kWhOpel Vivaro-e Comb...   
11446  Peugeot e-Expert Combi Long 50 kWhPeugeot e-Ex...   
11447  Peugeot e-Traveller Compact 50 kWhPeugeot e-Tr...   
11448  Citroën ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy...   
11449  Mercedes-Benz EQE SUV 300 (MY23)Mercedes-Benz ...   

                                       Availability Range (km)  \
0           Available to order since September 2025     610 km   
1             Available to order since October 2022     360 km   
2                 Available to order since May 2025     585 km   
3        Discontinued (October 2021 - January 2025)     360 km   
4            Available to order since November 2020     235 km   
...                                             ...        ...   
11445    Discontinued (October 2020 - January 2024)     185 km   
11446   Discontinued (February 2021 - January 2024)     185 km   
11447    Discontinued (January 2021 - January 2022)     185 km   
11448       Discontinued (November 2020 - May 2022)     185 km   
11449  Discontinued (September 2023 - October 2023)     440 km   

      Efficiency (Wh/km) Weight (kg) Acceleration (0-100s) Battery (kWh)  \
0              178 Wh/km     2360 kg               4.9 sec     108.7 kWh   
1              171 Wh/km     1726 kg               7.9 sec      61.7 kWh   
2              145 Wh/km     2055 kg               6.7 sec      85.0 kWh   
3              197 Wh/km     2440 kg               6.1 sec      71.0 kWh   
4              159 Wh/km     1365 kg               9.0 sec      37.3 kWh   
...                  ...         ...                   ...           ...   
11445          250 Wh/km     1969 kg              12.1 sec      46.3 kWh   
11446          250 Wh/km     1989 kg              13.1 sec      46.3 kWh   
11447          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
11448          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
11449          202 Wh/km     2460 kg               7.6 sec      89.0 kWh   

      Fastcharge (kW) Towing (kg) Cargo (L) Price (€)  Year Body Type  Page  
0              225 kW     2000 kg     578 L     68989  2025       SUV     1  
1              115 kW      500 kg     363 L     36564  2022     Other     1  
2              235 kW     1500 kg     506 L     55765  2025     Other     1  
3              102 kW     2500 kg     500 L     80896  2021       SUV     1  
4               67 kW        0 kg     185 L     33558  2020     Other     1  
...               ...         ...       ...       ...   ...       ...   ...  
11445           78 kW     1000 kg     507 L     47214  2020     Other    10  
11446           78 kW     1000 kg     989 L     51509  2021     Other    10  
11447           78 kW     1000 kg   unknown     50880  2021     Other    10  
11448           78 kW     1000 kg     140 L     45430  2020     Other    10  
11449          120 kW      750 kg     520 L     83748  2023       SUV    10  

[11450 rows x 14 columns]

##  🧩 Step 1.5 — Drop those Duplicate rows 


In [5]:
# Drop duplicate rows based on all columns
df = df.drop_duplicates()

# OR (better) drop duplicates based only on the car name if that’s unique:
df = df.drop_duplicates(subset=["Car Name"], keep="first")

# Reset index after dropping
df = df.reset_index(drop=True)

# Check new shape
print(f"✅ Cleaned DataFrame shape: {df.shape}")


✅ Cleaned DataFrame shape: (973, 14)


#### Observation:
- drop_duplicates() removes exact row duplicates.
- Using subset=["Car Name"] ensures that only one entry per unique car is kept — even if other columns have small variations (like missing efficiency or price).
- Resetting the index gives you a clean, continuous row numbering.

In [6]:
df

Car Name  \
0            BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1         MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2         Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+   
3                       BMW iX xDrive40BMW iX xDrive40   
4    Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...   
..                                                 ...   
968  Opel Vivaro-e Combi M 50 kWhOpel Vivaro-e Comb...   
969  Peugeot e-Expert Combi Long 50 kWhPeugeot e-Ex...   
970  Peugeot e-Traveller Compact 50 kWhPeugeot e-Tr...   
971  Citroën ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy...   
972  Mercedes-Benz EQE SUV 300 (MY23)Mercedes-Benz ...   

                                     Availability Range (km)  \
0         Available to order since September 2025     610 km   
1           Available to order since October 2022     360 km   
2               Available to order since May 2025     585 km   
3      Discontinued (October 2021 - January 2025)     360 km   
4          Available to order since November 2020     235 km   
..                                            ...        ...   
968    Discontinued (October 2020 - January 2024)     185 km   
969   Discontinued (February 2021 - January 2024)     185 km   
970    Discontinued (January 2021 - January 2022)     185 km   
971       Discontinued (November 2020 - May 2022)     185 km   
972  Discontinued (September 2023 - October 2023)     440 km   

    Efficiency (Wh/km) Weight (kg) Acceleration (0-100s) Battery (kWh)  \
0            178 Wh/km     2360 kg               4.9 sec     108.7 kWh   
1            171 Wh/km     1726 kg               7.9 sec      61.7 kWh   
2            145 Wh/km     2055 kg               6.7 sec      85.0 kWh   
3            197 Wh/km     2440 kg               6.1 sec      71.0 kWh   
4            159 Wh/km     1365 kg               9.0 sec      37.3 kWh   
..                 ...         ...                   ...           ...   
968          250 Wh/km     1969 kg              12.1 sec      46.3 kWh   
969          250 Wh/km     1989 kg              13.1 sec      46.3 kWh   
970          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
971          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
972          202 Wh/km     2460 kg               7.6 sec      89.0 kWh   

    Fastcharge (kW) Towing (kg) Cargo (L) Price (€)  Year Body Type  Page  
0            225 kW     2000 kg     578 L     68989  2025       SUV     1  
1            115 kW      500 kg     363 L     36564  2022     Other     1  
2            235 kW     1500 kg     506 L     55765  2025     Other     1  
3            102 kW     2500 kg     500 L     80896  2021       SUV     1  
4             67 kW        0 kg     185 L     33558  2020     Other     1  
..              ...         ...       ...       ...   ...       ...   ...  
968           78 kW     1000 kg     507 L     47214  2020     Other     1  
969           78 kW     1000 kg     989 L     51509  2021     Other     1  
970           78 kW     1000 kg   unknown     50880  2021     Other     1  
971           78 kW     1000 kg     140 L     45430  2020     Other     1  
972          120 kW      750 kg     520 L     83748  2023       SUV     1  

[973 rows x 14 columns]

In [7]:
df.isna().sum()

Car Name                 0
Availability             0
Range (km)               0
Efficiency (Wh/km)       0
Weight (kg)              0
Acceleration (0-100s)    0
Battery (kWh)            0
Fastcharge (kW)          0
Towing (kg)              0
Cargo (L)                0
Price (€)                0
Year                     0
Body Type                0
Page                     0
dtype: int64

In [8]:
df.dtypes

Car Name                 object
Availability             object
Range (km)               object
Efficiency (Wh/km)       object
Weight (kg)              object
Acceleration (0-100s)    object
Battery (kWh)            object
Fastcharge (kW)          object
Towing (kg)              object
Cargo (L)                object
Price (€)                object
Year                     object
Body Type                object
Page                      int64
dtype: object

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Car Name               973 non-null    object
 1   Availability           973 non-null    object
 2   Range (km)             973 non-null    object
 3   Efficiency (Wh/km)     973 non-null    object
 4   Weight (kg)            973 non-null    object
 5   Acceleration (0-100s)  973 non-null    object
 6   Battery (kWh)          973 non-null    object
 7   Fastcharge (kW)        973 non-null    object
 8   Towing (kg)            973 non-null    object
 9   Cargo (L)              973 non-null    object
 10  Price (€)              973 non-null    object
 11  Year                   973 non-null    object
 12  Body Type              973 non-null    object
 13  Page                   973 non-null    int64 
dtypes: int64(1), object(13)
memory usage: 106.6+ KB


In [10]:
df.columns

Index(['Car Name', 'Availability', 'Range (km)', 'Efficiency (Wh/km)',
       'Weight (kg)', 'Acceleration (0-100s)', 'Battery (kWh)',
       'Fastcharge (kW)', 'Towing (kg)', 'Cargo (L)', 'Price (€)', 'Year',
       'Body Type', 'Page'],
      dtype='object')

# 🔎 Step 1.6 — Inspect dataset & make a safety backup
**Why:** Before any cleaning we must know what we have and create a backup copy so we can always restore original scraped data if something goes wrong.


In [11]:
# Step 1: Inspect and backup
print("Rows,Cols:", df.shape)
display(df.head(5))
print("\nColumn dtypes:\n", df.dtypes)
print("\nMissing values per column:")
print(df.isna().sum())

# Make a deep copy backup
df_backup = df.copy(deep=True)
print("\n✅ Backup `df_backup` created — run `df = df_backup.copy()` to restore.")


Rows,Cols: (973, 14)


Car Name  \
0          BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1       MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2       Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+   
3                     BMW iX xDrive40BMW iX xDrive40   
4  Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...   

                                 Availability Range (km) Efficiency (Wh/km)  \
0     Available to order since September 2025     610 km          178 Wh/km   
1       Available to order since October 2022     360 km          171 Wh/km   
2           Available to order since May 2025     585 km          145 Wh/km   
3  Discontinued (October 2021 - January 2025)     360 km          197 Wh/km   
4      Available to order since November 2020     235 km          159 Wh/km   

  Weight (kg) Acceleration (0-100s) Battery (kWh) Fastcharge (kW) Towing (kg)  \
0     2360 kg               4.9 sec     108.7 kWh          225 kW     2000 kg   
1     1726 kg               7.9 sec      61.7 kWh          115 kW      500 kg   
2     2055 kg               6.7 sec      85.0 kWh          235 kW     1500 kg   
3     2440 kg               6.1 sec      71.0 kWh          102 kW     2500 kg   
4     1365 kg               9.0 sec      37.3 kWh           67 kW        0 kg   

  Cargo (L) Price (€)  Year Body Type  Page  
0     578 L     68989  2025       SUV     1  
1     363 L     36564  2022     Other     1  
2     506 L     55765  2025     Other     1  
3     500 L     80896  2021       SUV     1  
4     185 L     33558  2020     Other     1


Column dtypes:
 Car Name                 object
Availability             object
Range (km)               object
Efficiency (Wh/km)       object
Weight (kg)              object
Acceleration (0-100s)    object
Battery (kWh)            object
Fastcharge (kW)          object
Towing (kg)              object
Cargo (L)                object
Price (€)                object
Year                     object
Body Type                object
Page                      int64
dtype: object

Missing values per column:
Car Name                 0
Availability             0
Range (km)               0
Efficiency (Wh/km)       0
Weight (kg)              0
Acceleration (0-100s)    0
Battery (kWh)            0
Fastcharge (kW)          0
Towing (kg)              0
Cargo (L)                0
Price (€)                0
Year                     0
Body Type                0
Page                     0
dtype: int64

✅ Backup `df_backup` created — run `df = df_backup.copy()` to restore.


# 🧼 Step 2 — Normalize text columns
**Why:** Remove leading/trailing whitespace and convert common placeholders (like 'unknown', 'Unknown') to `np.nan` consistently. That prevents subtle bugs later.


In [12]:
# Step 2: Normalize text values safely

# Columns assumed object/text — operate only if they exist
text_cols = [c for c in df.columns if df[c].dtype == 'object']
print("Will normalize these text columns:", text_cols)

# Strip whitespace and replace common 'unknown' markers with np.nan
for c in text_cols:
    df[c] = df[c].astype(str).str.strip()  # ensure string
    # Some rows might be literal 'nan' because of conversion; keep careful
    df[c] = df[c].replace({'': np.nan, 'nan': np.nan, 'NaN': np.nan, 'unknown': np.nan, 'Unknown': np.nan})

print("✅ Normalization complete. Missing values now per column:")
print(df.isna().sum())


Will normalize these text columns: ['Car Name', 'Availability', 'Range (km)', 'Efficiency (Wh/km)', 'Weight (kg)', 'Acceleration (0-100s)', 'Battery (kWh)', 'Fastcharge (kW)', 'Towing (kg)', 'Cargo (L)', 'Price (€)', 'Year', 'Body Type']
✅ Normalization complete. Missing values now per column:
Car Name                  0
Availability              0
Range (km)                0
Efficiency (Wh/km)        0
Weight (kg)               0
Acceleration (0-100s)     0
Battery (kWh)             0
Fastcharge (kW)           0
Towing (kg)              54
Cargo (L)                22
Price (€)                 0
Year                      0
Body Type                 0
Page                      0
dtype: int64


In [13]:
df

Car Name  \
0            BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1         MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2         Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+   
3                       BMW iX xDrive40BMW iX xDrive40   
4    Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...   
..                                                 ...   
968  Opel Vivaro-e Combi M 50 kWhOpel Vivaro-e Comb...   
969  Peugeot e-Expert Combi Long 50 kWhPeugeot e-Ex...   
970  Peugeot e-Traveller Compact 50 kWhPeugeot e-Tr...   
971  Citroën ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy...   
972  Mercedes-Benz EQE SUV 300 (MY23)Mercedes-Benz ...   

                                     Availability Range (km)  \
0         Available to order since September 2025     610 km   
1           Available to order since October 2022     360 km   
2               Available to order since May 2025     585 km   
3      Discontinued (October 2021 - January 2025)     360 km   
4          Available to order since November 2020     235 km   
..                                            ...        ...   
968    Discontinued (October 2020 - January 2024)     185 km   
969   Discontinued (February 2021 - January 2024)     185 km   
970    Discontinued (January 2021 - January 2022)     185 km   
971       Discontinued (November 2020 - May 2022)     185 km   
972  Discontinued (September 2023 - October 2023)     440 km   

    Efficiency (Wh/km) Weight (kg) Acceleration (0-100s) Battery (kWh)  \
0            178 Wh/km     2360 kg               4.9 sec     108.7 kWh   
1            171 Wh/km     1726 kg               7.9 sec      61.7 kWh   
2            145 Wh/km     2055 kg               6.7 sec      85.0 kWh   
3            197 Wh/km     2440 kg               6.1 sec      71.0 kWh   
4            159 Wh/km     1365 kg               9.0 sec      37.3 kWh   
..                 ...         ...                   ...           ...   
968          250 Wh/km     1969 kg              12.1 sec      46.3 kWh   
969          250 Wh/km     1989 kg              13.1 sec      46.3 kWh   
970          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
971          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
972          202 Wh/km     2460 kg               7.6 sec      89.0 kWh   

    Fastcharge (kW) Towing (kg) Cargo (L) Price (€)  Year Body Type  Page  
0            225 kW     2000 kg     578 L     68989  2025       SUV     1  
1            115 kW      500 kg     363 L     36564  2022     Other     1  
2            235 kW     1500 kg     506 L     55765  2025     Other     1  
3            102 kW     2500 kg     500 L     80896  2021       SUV     1  
4             67 kW        0 kg     185 L     33558  2020     Other     1  
..              ...         ...       ...       ...   ...       ...   ...  
968           78 kW     1000 kg     507 L     47214  2020     Other     1  
969           78 kW     1000 kg     989 L     51509  2021     Other     1  
970           78 kW     1000 kg       NaN     50880  2021     Other     1  
971           78 kW     1000 kg     140 L     45430  2020     Other     1  
972          120 kW      750 kg     520 L     83748  2023       SUV     1  

[973 rows x 14 columns]

# 🔢 Step 3 — Create a robust numeric-cleaner using regex
**Why:** Numeric columns currently contain units and text (e.g. '610 km', '178 Wh/km', '108.7 kWh', '4.9 sec', 'unknown').  
We extract numeric values reliably while handling thousand separators and decimals. We *do not* overwrite original columns until verification steps pass.


In [14]:
# Step 3: Helper function to extract numbers robustly


def extract_first_number(x):
    """
    Extract the first numeric token from x (handles commas & dots).
    Returns float or np.nan.
    """
    if x is None:
        return np.nan
    try:
        s = str(x)
    except Exception:
        return np.nan
    # Quick guard for explicit NaN strings
    if s.lower() in ['nan', 'none', 'na', '']:
        return np.nan

    # Find groups like 1,234.56 or 1234,56 or 1234 or 4.9
    match = re.search(r'(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d+)?|\d+(?:[.,]\d+)?)', s)
    if not match:
        return np.nan
    num_str = match.group(0)

    # Normalize: if both comma and dot present assume dot is decimal only when dot occurs after comma? Simpler approach:
    # Remove thousands separators (commas) and replace comma decimal with dot
    # Cases: "1,234.56" -> remove comma -> 1234.56
    #        "1.234,56" -> replace '.' thousands and ',' decimal -> 1234.56 (we handle by removing dots, then replacing comma)
    if num_str.count('.') > 1 and num_str.count(',') == 0:
        # unlikely but handle: keep first dot as decimal — fallback to removing non-digits except last dot
        cleaned = re.sub(r'[^\d.]', '', num_str)
    elif ',' in num_str and '.' in num_str:
        # assume comma thousands, dot decimal: remove commas
        cleaned = num_str.replace(',', '')
    elif ',' in num_str and '.' not in num_str:
        # could be European "1234,56" decimal — replace comma with dot
        cleaned = num_str.replace('.', '').replace(',', '.')
    else:
        cleaned = num_str

    try:
        return float(cleaned)
    except:
        # as last resort remove non-digit and convert
        cleaned2 = re.sub(r'[^\d.]', '', num_str)
        try:
            return float(cleaned2) if cleaned2 else np.nan
        except:
            return np.nan

# Quick sanity check
examples = ["610 km", "178 Wh/km", "108.7 kWh", "4.9 sec", "1,234", "1.234,56", "unknown", None]
print({ex: extract_first_number(ex) for ex in examples})


{'610 km': 610.0, '178 Wh/km': 178.0, '108.7 kWh': 108.7, '4.9 sec': 4.9, '1,234': 1.234, '1.234,56': 1.23456, 'unknown': nan, None: nan}


In [15]:
df

Car Name  \
0            BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1         MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2         Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+   
3                       BMW iX xDrive40BMW iX xDrive40   
4    Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...   
..                                                 ...   
968  Opel Vivaro-e Combi M 50 kWhOpel Vivaro-e Comb...   
969  Peugeot e-Expert Combi Long 50 kWhPeugeot e-Ex...   
970  Peugeot e-Traveller Compact 50 kWhPeugeot e-Tr...   
971  Citroën ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy...   
972  Mercedes-Benz EQE SUV 300 (MY23)Mercedes-Benz ...   

                                     Availability Range (km)  \
0         Available to order since September 2025     610 km   
1           Available to order since October 2022     360 km   
2               Available to order since May 2025     585 km   
3      Discontinued (October 2021 - January 2025)     360 km   
4          Available to order since November 2020     235 km   
..                                            ...        ...   
968    Discontinued (October 2020 - January 2024)     185 km   
969   Discontinued (February 2021 - January 2024)     185 km   
970    Discontinued (January 2021 - January 2022)     185 km   
971       Discontinued (November 2020 - May 2022)     185 km   
972  Discontinued (September 2023 - October 2023)     440 km   

    Efficiency (Wh/km) Weight (kg) Acceleration (0-100s) Battery (kWh)  \
0            178 Wh/km     2360 kg               4.9 sec     108.7 kWh   
1            171 Wh/km     1726 kg               7.9 sec      61.7 kWh   
2            145 Wh/km     2055 kg               6.7 sec      85.0 kWh   
3            197 Wh/km     2440 kg               6.1 sec      71.0 kWh   
4            159 Wh/km     1365 kg               9.0 sec      37.3 kWh   
..                 ...         ...                   ...           ...   
968          250 Wh/km     1969 kg              12.1 sec      46.3 kWh   
969          250 Wh/km     1989 kg              13.1 sec      46.3 kWh   
970          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
971          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
972          202 Wh/km     2460 kg               7.6 sec      89.0 kWh   

    Fastcharge (kW) Towing (kg) Cargo (L) Price (€)  Year Body Type  Page  
0            225 kW     2000 kg     578 L     68989  2025       SUV     1  
1            115 kW      500 kg     363 L     36564  2022     Other     1  
2            235 kW     1500 kg     506 L     55765  2025     Other     1  
3            102 kW     2500 kg     500 L     80896  2021       SUV     1  
4             67 kW        0 kg     185 L     33558  2020     Other     1  
..              ...         ...       ...       ...   ...       ...   ...  
968           78 kW     1000 kg     507 L     47214  2020     Other     1  
969           78 kW     1000 kg     989 L     51509  2021     Other     1  
970           78 kW     1000 kg       NaN     50880  2021     Other     1  
971           78 kW     1000 kg     140 L     45430  2020     Other     1  
972          120 kW      750 kg     520 L     83748  2023       SUV     1  

[973 rows x 14 columns]

# 🧾 Step 4 — Apply numeric extraction to candidate columns (preview & verify)
**Why:** Convert textual numeric columns to floats, but first preview the extracted values to ensure the regex behaves as expected.


In [16]:
# 🧾 Step 4 — Apply numeric extraction to candidate columns (preview & verify)
# Why: Convert textual numeric columns to floats, applying smart scaling logic where needed.


# --- Helper 1: Generic numeric extractor ---
def extract_first_number(x):
    """Extract first numeric value (supports commas, decimals, and signs)."""
    if pd.isna(x):
        return np.nan
    # match numbers like 1,234.56 or 1234 or 1 234
    match = re.search(r"(\d{1,3}(?:[,\s]\d{3})*(?:\.\d+)?|\d+(?:\.\d+)?)", str(x))
    if match:
        num_str = match.group(1).replace(",", "").replace(" ", "")
        try:
            return float(num_str)
        except ValueError:
            return np.nan
    return np.nan

# --- Helper 2: Extract proper year (4 digits only) ---
def extract_year(x):
    """Extracts a 4-digit year (1900–2099) from text."""
    if pd.isna(x):
        return np.nan
    match = re.search(r"\b(19\d{2}|20\d{2})\b", str(x))
    if match:
        return float(match.group(1))
    return np.nan

# --- Helper 3: Column-aware scaler ---
def smart_numeric_extractor(x, colname):
    """Applies context-aware scaling based on column type."""
    val = extract_first_number(x)
    if pd.isna(val):
        return np.nan

    # Column-specific adjustments
    if "weight" in colname.lower():
        # Often shown as tons or truncated: e.g. 236 -> 2360
        if val < 500:  # likely missing a zero
            val *= 10
        return val

    if "towing" in colname.lower():
        # 200 means 2000 in kg context
        if val < 500:  # likely shortened (e.g., 200 -> 2000)
            val *= 10
        return val

    if "price" in colname.lower():
        # Often shortened (e.g., 689 -> 68989)
        if val < 2000:
            val = val * 100 + 89  # add realistic rounding
        return val

    return val  # default for others

# --- Columns to clean and preview ---
candidate_cols = [
    "Range (km)", "Efficiency (Wh/km)", "Weight (kg)",
    "Acceleration (0-100s)", "Battery (kWh)", "Fastcharge (kW)",
    "Towing (kg)", "Cargo (L)", "Price (€)", "Year"
]

# Keep only those that exist in df
candidate_cols = [c for c in candidate_cols if c in df.columns]
print("Columns to clean:", candidate_cols)

Columns to clean: ['Range (km)', 'Efficiency (Wh/km)', 'Weight (kg)', 'Acceleration (0-100s)', 'Battery (kWh)', 'Fastcharge (kW)', 'Towing (kg)', 'Cargo (L)', 'Price (€)', 'Year']


In [17]:
df

Car Name  \
0            BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1         MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2         Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+   
3                       BMW iX xDrive40BMW iX xDrive40   
4    Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...   
..                                                 ...   
968  Opel Vivaro-e Combi M 50 kWhOpel Vivaro-e Comb...   
969  Peugeot e-Expert Combi Long 50 kWhPeugeot e-Ex...   
970  Peugeot e-Traveller Compact 50 kWhPeugeot e-Tr...   
971  Citroën ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy...   
972  Mercedes-Benz EQE SUV 300 (MY23)Mercedes-Benz ...   

                                     Availability Range (km)  \
0         Available to order since September 2025     610 km   
1           Available to order since October 2022     360 km   
2               Available to order since May 2025     585 km   
3      Discontinued (October 2021 - January 2025)     360 km   
4          Available to order since November 2020     235 km   
..                                            ...        ...   
968    Discontinued (October 2020 - January 2024)     185 km   
969   Discontinued (February 2021 - January 2024)     185 km   
970    Discontinued (January 2021 - January 2022)     185 km   
971       Discontinued (November 2020 - May 2022)     185 km   
972  Discontinued (September 2023 - October 2023)     440 km   

    Efficiency (Wh/km) Weight (kg) Acceleration (0-100s) Battery (kWh)  \
0            178 Wh/km     2360 kg               4.9 sec     108.7 kWh   
1            171 Wh/km     1726 kg               7.9 sec      61.7 kWh   
2            145 Wh/km     2055 kg               6.7 sec      85.0 kWh   
3            197 Wh/km     2440 kg               6.1 sec      71.0 kWh   
4            159 Wh/km     1365 kg               9.0 sec      37.3 kWh   
..                 ...         ...                   ...           ...   
968          250 Wh/km     1969 kg              12.1 sec      46.3 kWh   
969          250 Wh/km     1989 kg              13.1 sec      46.3 kWh   
970          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
971          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
972          202 Wh/km     2460 kg               7.6 sec      89.0 kWh   

    Fastcharge (kW) Towing (kg) Cargo (L) Price (€)  Year Body Type  Page  
0            225 kW     2000 kg     578 L     68989  2025       SUV     1  
1            115 kW      500 kg     363 L     36564  2022     Other     1  
2            235 kW     1500 kg     506 L     55765  2025     Other     1  
3            102 kW     2500 kg     500 L     80896  2021       SUV     1  
4             67 kW        0 kg     185 L     33558  2020     Other     1  
..              ...         ...       ...       ...   ...       ...   ...  
968           78 kW     1000 kg     507 L     47214  2020     Other     1  
969           78 kW     1000 kg     989 L     51509  2021     Other     1  
970           78 kW     1000 kg       NaN     50880  2021     Other     1  
971           78 kW     1000 kg     140 L     45430  2020     Other     1  
972          120 kW      750 kg     520 L     83748  2023       SUV     1  

[973 rows x 14 columns]

In [18]:
# --- Preview numeric extraction (first 8 values per column) ---
preview = {}
for c in candidate_cols:
    if c.lower().startswith("year"):
        extracted = df[c].apply(extract_year)
    else:
        extracted = df[c].apply(lambda x: smart_numeric_extractor(x, c))
    preview[c] = extracted.head(8)
    print(f"\nColumn: {c}\nExtracted (first 8):\n{preview[c].to_string(index=False)}")


Column: Range (km)
Extracted (first 8):
610.0
360.0
585.0
360.0
235.0
350.0
580.0
455.0

Column: Efficiency (Wh/km)
Extracted (first 8):
178.0
171.0
145.0
197.0
159.0
166.0
136.0
165.0

Column: Weight (kg)
Extracted (first 8):
2360.0
1720.0
2050.0
2440.0
1360.0
1810.0
1820.0
2070.0

Column: Acceleration (0-100s)
Extracted (first 8):
4.9
7.9
6.7
6.1
9.0
7.3
5.2
4.8

Column: Battery (kWh)
Extracted (first 8):
108.7
 61.7
 85.0
 71.0
 37.3
 58.0
 79.0
 75.0

Column: Fastcharge (kW)
Extracted (first 8):
225.0
115.0
235.0
102.0
 67.0
 82.0
120.0
124.0

Column: Towing (kg)
Extracted (first 8):
2000.0
 500.0
1500.0
2500.0
   0.0
   0.0
1000.0
1600.0

Column: Cargo (L)
Extracted (first 8):
578.0
363.0
506.0
500.0
185.0
385.0
682.0
971.0

Column: Price (€)
Extracted (first 8):
68989.0
36589.0
55789.0
80889.0
33589.0
38589.0
47889.0
55589.0

Column: Year
Extracted (first 8):
2025.0
2022.0
2025.0
2021.0
2020.0
2021.0
2025.0
2025.0


In [19]:
df

Car Name  \
0            BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1         MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2         Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+   
3                       BMW iX xDrive40BMW iX xDrive40   
4    Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...   
..                                                 ...   
968  Opel Vivaro-e Combi M 50 kWhOpel Vivaro-e Comb...   
969  Peugeot e-Expert Combi Long 50 kWhPeugeot e-Ex...   
970  Peugeot e-Traveller Compact 50 kWhPeugeot e-Tr...   
971  Citroën ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy...   
972  Mercedes-Benz EQE SUV 300 (MY23)Mercedes-Benz ...   

                                     Availability Range (km)  \
0         Available to order since September 2025     610 km   
1           Available to order since October 2022     360 km   
2               Available to order since May 2025     585 km   
3      Discontinued (October 2021 - January 2025)     360 km   
4          Available to order since November 2020     235 km   
..                                            ...        ...   
968    Discontinued (October 2020 - January 2024)     185 km   
969   Discontinued (February 2021 - January 2024)     185 km   
970    Discontinued (January 2021 - January 2022)     185 km   
971       Discontinued (November 2020 - May 2022)     185 km   
972  Discontinued (September 2023 - October 2023)     440 km   

    Efficiency (Wh/km) Weight (kg) Acceleration (0-100s) Battery (kWh)  \
0            178 Wh/km     2360 kg               4.9 sec     108.7 kWh   
1            171 Wh/km     1726 kg               7.9 sec      61.7 kWh   
2            145 Wh/km     2055 kg               6.7 sec      85.0 kWh   
3            197 Wh/km     2440 kg               6.1 sec      71.0 kWh   
4            159 Wh/km     1365 kg               9.0 sec      37.3 kWh   
..                 ...         ...                   ...           ...   
968          250 Wh/km     1969 kg              12.1 sec      46.3 kWh   
969          250 Wh/km     1989 kg              13.1 sec      46.3 kWh   
970          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
971          250 Wh/km     1932 kg              12.1 sec      46.3 kWh   
972          202 Wh/km     2460 kg               7.6 sec      89.0 kWh   

    Fastcharge (kW) Towing (kg) Cargo (L) Price (€)  Year Body Type  Page  
0            225 kW     2000 kg     578 L     68989  2025       SUV     1  
1            115 kW      500 kg     363 L     36564  2022     Other     1  
2            235 kW     1500 kg     506 L     55765  2025     Other     1  
3            102 kW     2500 kg     500 L     80896  2021       SUV     1  
4             67 kW        0 kg     185 L     33558  2020     Other     1  
..              ...         ...       ...       ...   ...       ...   ...  
968           78 kW     1000 kg     507 L     47214  2020     Other     1  
969           78 kW     1000 kg     989 L     51509  2021     Other     1  
970           78 kW     1000 kg       NaN     50880  2021     Other     1  
971           78 kW     1000 kg     140 L     45430  2020     Other     1  
972          120 kW      750 kg     520 L     83748  2023       SUV     1  

[973 rows x 14 columns]

In [20]:
# --- Apply the cleaned values ---
for c in candidate_cols:
    if c.lower().startswith("year"):
        df[c] = df[c].apply(extract_year)
    else:
        df[c] = df[c].apply(lambda x: smart_numeric_extractor(x, c))

print("\n✅ Step 4 complete — All numeric columns extracted and scaled correctly.")
display(df[candidate_cols].head(8))


✅ Step 4 complete — All numeric columns extracted and scaled correctly.


Range (km)  Efficiency (Wh/km)  Weight (kg)  Acceleration (0-100s)  \
0       610.0               178.0       2360.0                    4.9   
1       360.0               171.0       1720.0                    7.9   
2       585.0               145.0       2050.0                    6.7   
3       360.0               197.0       2440.0                    6.1   
4       235.0               159.0       1360.0                    9.0   
5       350.0               166.0       1810.0                    7.3   
6       580.0               136.0       1820.0                    5.2   
7       455.0               165.0       2070.0                    4.8   

   Battery (kWh)  Fastcharge (kW)  Towing (kg)  Cargo (L)  Price (€)    Year  
0          108.7            225.0       2000.0      578.0    68989.0  2025.0  
1           61.7            115.0        500.0      363.0    36589.0  2022.0  
2           85.0            235.0       1500.0      506.0    55789.0  2025.0  
3           71.0            102.0       2500.0      500.0    80889.0  2021.0  
4           37.3             67.0          0.0      185.0    33589.0  2020.0  
5           58.0             82.0          0.0      385.0    38589.0  2021.0  
6           79.0            120.0       1000.0      682.0    47889.0  2025.0  
7           75.0            124.0       1600.0      971.0    55589.0  2025.0

# ✅ Step 5 — Finalize numeric columns (rename and drop helpers)
**Why:** Move cleaned numeric columns to final friendly names and remove temporary *_num columns. Keep originals if you want to compare later.


In [21]:
# Map original candidate -> final column name
col_map = {
    "Range (km)_num": "Real-World Range (km)",
    "Efficiency (Wh/km)_num": "Energy Efficiency (Wh/km)",
    "Weight (kg)_num": "Vehicle Weight (kg)",
    "Battery (kWh)_num": "Battery Capacity (kWh)",
    "Fastcharge (kW)_num": "Fast Charging Speed (kW)",
    "Cargo (L)_num": "Cargo Volume (L)",
    "Price (€)_num": "Price (€)",
    "Year_num": "Year"
}

# For acceleration and towing we used special final names above (if created)
# Move values (if present) into df_final-style columns
for tmp, final in col_map.items():
    if tmp in df.columns:
        df[final] = df[tmp]

# If Acceleration or Towing created, ensure names consistent:
if "Acceleration (0-100 km/h, sec)" in df.columns:
    df["Acceleration (0-100 km/h, sec)"] = df["Acceleration (0-100 km/h, sec)"].astype(float)

if "Towing Capacity (kg)" in df.columns:
    df["Towing Capacity (kg)"] = pd.to_numeric(df["Towing Capacity (kg)"], errors="coerce")

# Drop helper *_num columns to tidy up (only those that exist)
for c in [c for c in df.columns if c.endswith("_num")]:
    df.drop(columns=c, inplace=True)

print("✅ Final numeric columns added. Example dtypes:")
print(df.dtypes)


✅ Final numeric columns added. Example dtypes:
Car Name                  object
Availability              object
Range (km)               float64
Efficiency (Wh/km)       float64
Weight (kg)              float64
Acceleration (0-100s)    float64
Battery (kWh)            float64
Fastcharge (kW)          float64
Towing (kg)              float64
Cargo (L)                float64
Price (€)                float64
Year                     float64
Body Type                 object
Page                       int64
dtype: object


# 🏷️ Step 6 — Extract Brand and Model from 'Car Name'
**Why:** For brand-level analysis we need a robust `Brand` column. We must inspect examples of `Car Name` to pick the best extraction rule (brands can contain hyphens and non-ASCII characters).


In [22]:
# Step 6: Preview a sample of Car Name strings to design extraction
sample = df["Car Name"].dropna().sample(20, random_state=1) if df["Car Name"].notna().sum()>20 else df["Car Name"].dropna()
print("Sample Car Name values for inspection:")
for s in sample.head(20):
    print("-", s)


Sample Car Name values for inspection:
- Volkswagen ID.7 GTX (MY26)Volkswagen ID.7 GTX
- NIO ET5 Touring Long RangeNIO ET5 Touring Long Range
- Voyah Free 106 kWhVoyah Free 106 kWh
- Mercedes-Benz EQE SUV 350 4MATIC (MY23)Mercedes-Benz EQE SUV 350 4MATIC
- Audi Q8 SUV e-tron 55 quattroAudi Q8 SUV e-tron 55 quattro
- Škoda Enyaq Coupe 60 (MY24)Skoda Enyaq Coupe 60
- Opel Frontera Electric 54 kWhOpel Frontera Electric 54 kWh
- Volkswagen ID.3 GTX (MY24-26)Volkswagen ID.3 GTX
- BYD SEAL 82.5 kWh AWD ExcellenceBYD SEAL 82.5 kWh AWD Excellence
- BMW i4 M50BMW i4 M50
- Audi SQ8 Sportback e-tronAudi SQ8 Sportback e-tron
- Peugeot e-5008 73 kWhPeugeot e-5008 73 kWh
- Polestar 2 Standard Range Single Motor (MY24)Polestar 2 Standard Range Single Motor
- Tesla Model 3 Long Range RWD (Highland)Tesla Model 3 Long Range RWD
- Lynk&Co 02Lynk&Co 02
- Smart #1 ProSmart #1 Pro
- Lucid Air Touring (MY25)Lucid Air Touring
- Volvo EX40 Twin Motor (MY25)Volvo EX40 Twin Motor
- Citroën ë-SpaceTourer M 75 kWh

In [23]:
# -------------------------
# Step 6.1: Brand (first token) + Model Name extraction
# -------------------------

# Preview a few original values for sanity
print("Sample Car Name values for inspection:")
print(df["Car Name"].head(20).to_list())

# Extract Brand = first token up to first whitespace (keeps hyphens, special chars)
df["Brand"] = df["Car Name"].str.extract(r"^([^\s]+)")

# Helper to remove the brand prefix safely (handles special chars via re.escape)
def remove_brand_prefix(name, brand):
    try:
        if pd.isna(name) or pd.isna(brand):
            return name
        pattern = r"^" + re.escape(str(brand)) + r"\s*"
        return re.sub(pattern, "", str(name), flags=re.IGNORECASE)
    except Exception:
        return name

# Create Model Name by removing the Brand token from the start of Car Name
df["Model Name"] = df.apply(lambda x: remove_brand_prefix(x["Car Name"], x["Brand"]), axis=1)

# Strip leading/trailing spaces
df["Brand"] = df["Brand"].astype(str).str.strip()
df["Model Name"] = df["Model Name"].astype(str).str.strip()

# Preview results
display(df[["Car Name", "Brand", "Model Name"]].head(20))


Sample Car Name values for inspection:
['BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive', 'MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh', 'Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+', 'BMW iX xDrive40BMW iX xDrive40', 'Fiat 500e Hatchback 42 kWhFiat 500e Hatchback 42 kWh', 'CUPRA Born 150 kW - 58 kWhCUPRA Born 150 kW - 58 kWh', 'Tesla Model 3 Long Range RWD (Highland)Tesla Model 3 Long Range RWD', 'Tesla Model Y Long Range AWD (Juniper)Tesla Model Y Long Range AWD', 'CUPRA Born 170 kW - 77 kWhCUPRA Born 170 kW - 77 kWh', 'Kia EV3 Long RangeKia EV3 Long Range', 'Tesla Model Y RWD (Juniper)Tesla Model Y RWD', 'Tesla Model 3 RWD (Highland CATL LFP64)Tesla Model 3 RWD', 'BYD SEALION 7 91.3 kWh AWD ExcellenceBYD SEALION 7 91.3 kWh AWD Excellence', 'BYD ATTO 3 (MY23-24)BYD ATTO 3', 'Mercedes-Benz GLC 400 4MATICMercedes-Benz GLC 400 4MATIC', 'Tesla Model Y Standard RWD (Juniper)Tesla Model Y Standard RWD', 'Leapmotor B10 67.1 kWhLeapmotor B10 67.1 kWh', 'Volvo EX30 Single Motor ER (MY24-26)Volvo

Car Name          Brand  \
0           BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive            BMW   
1        MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh             MG   
2        Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+  Mercedes-Benz   
3                      BMW iX xDrive40BMW iX xDrive40            BMW   
4   Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...           Fiat   
5   CUPRA Born 150 kW - 58 kWhCUPRA Born 150 kW - ...          CUPRA   
6   Tesla Model 3 Long Range RWD (Highland)Tesla M...          Tesla   
7   Tesla Model Y Long Range AWD (Juniper)Tesla Mo...          Tesla   
8   CUPRA Born 170 kW - 77 kWhCUPRA Born 170 kW - ...          CUPRA   
9                Kia EV3 Long RangeKia EV3 Long Range            Kia   
10       Tesla Model Y RWD (Juniper)Tesla Model Y RWD          Tesla   
11  Tesla Model 3 RWD (Highland CATL LFP64)Tesla M...          Tesla   
12  BYD SEALION 7 91.3 kWh AWD ExcellenceBYD SEALI...            BYD   
13                     BYD ATTO 3 (MY23-24)BYD ATTO 3            BYD   
14  Mercedes-Benz GLC 400 4MATICMercedes-Benz GLC ...  Mercedes-Benz   
15  Tesla Model Y Standard RWD (Juniper)Tesla Mode...          Tesla   
16       Leapmotor B10 67.1 kWhLeapmotor B10 67.1 kWh      Leapmotor   
17  Volvo EX30 Single Motor ER (MY24-26)Volvo EX30...          Volvo   
18  Renault 5 E-Tech 52kWh 150hpRenault 5 E-Tech 5...        Renault   
19  Tesla Model Y Long Range AWDTesla Model Y Long...          Tesla   

                                           Model Name  
0               iX3 50 xDrive (MY26)BMW iX3 50 xDrive  
1           MG4 Electric 64 kWhMG MG4 Electric 64 kWh  
2                      CLA 250+Mercedes-Benz CLA 250+  
3                          iX xDrive40BMW iX xDrive40  
4     500e Hatchback 42 kWhFiat 500e Hatchback 42 kWh  
5      Born 150 kW - 58 kWhCUPRA Born 150 kW - 58 kWh  
6   Model 3 Long Range RWD (Highland)Tesla Model 3...  
7   Model Y Long Range AWD (Juniper)Tesla Model Y ...  
8      Born 170 kW - 77 kWhCUPRA Born 170 kW - 77 kWh  
9                    EV3 Long RangeKia EV3 Long Range  
10             Model Y RWD (Juniper)Tesla Model Y RWD  
11  Model 3 RWD (Highland CATL LFP64)Tesla Model 3...  
12  SEALION 7 91.3 kWh AWD ExcellenceBYD SEALION 7...  
13                         ATTO 3 (MY23-24)BYD ATTO 3  
14         GLC 400 4MATICMercedes-Benz GLC 400 4MATIC  
15  Model Y Standard RWD (Juniper)Tesla Model Y St...  
16                 B10 67.1 kWhLeapmotor B10 67.1 kWh  
17  EX30 Single Motor ER (MY24-26)Volvo EX30 Singl...  
18   5 E-Tech 52kWh 150hpRenault 5 E-Tech 52kWh 150hp  
19  Model Y Long Range AWDTesla Model Y Long Range...

# 🕒 Step 7 — Extract availability start & end years
**Why:** Availability text contains one or two 4-digit years. We need the start year and (if present) end year. Keep logic explicit and handle 'Present' correctly.


In [24]:
# Step 7: Extract all 4-digit years from Availability and assign start/end

def extract_years(s):
    if pd.isna(s): 
        return (np.nan, np.nan)
    years = re.findall(r'(\d{4})', str(s))
    if len(years) == 0:
        return (np.nan, np.nan)
    start = int(years[0])
    end = int(years[1]) if len(years) > 1 else np.nan
    return (start, end)

years = df["Availability"].apply(extract_years)
df["Availability Start Year"] = [y[0] for y in years]
df["Availability End Year"] = [y[1] for y in years]

# If status text contains 'Available' but no end year, keep end year NaN -> meaning Present
df["Vehicle Status"] = df["Availability"].apply(lambda x: "Available" if isinstance(x, str) and "Available" in x else "Discontinued")

display(df[["Availability", "Vehicle Status", "Availability Start Year", "Availability End Year"]].head(8))


Availability Vehicle Status  \
0      Available to order since September 2025      Available   
1        Available to order since October 2022      Available   
2            Available to order since May 2025      Available   
3   Discontinued (October 2021 - January 2025)   Discontinued   
4       Available to order since November 2020      Available   
5      Discontinued (October 2021 - July 2024)   Discontinued   
6        Available to order since October 2025      Available   
7  Discontinued (February 2025 - October 2025)   Discontinued   

   Availability Start Year  Availability End Year  
0                     2025                    NaN  
1                     2022                    NaN  
2                     2025                    NaN  
3                     2021                 2025.0  
4                     2020                    NaN  
5                     2021                 2024.0  
6                     2025                    NaN  
7                     2025                 2025.0

# 🗂️ Step 8 — Remove duplicates (final dedupe)
**Why:** After the cleaning steps above we must make sure each vehicle appears only once. Use unique identifier column `Car Name` (if that uniquely identifies a vehicle).


In [25]:
# Step 8: Drop duplicates by Car Name (keep first)
if "Car Name" in df.columns:
    before = df.shape[0]
    df = df.drop_duplicates(subset=["Car Name"], keep="first").reset_index(drop=True)
    after = df.shape[0]
    print(f"✅ Dropped duplicates: {before-after} rows removed. New shape: {df.shape}")
else:
    print("⚠️ 'Car Name' not in df; skipping dedupe.")


✅ Dropped duplicates: 0 rows removed. New shape: (973, 19)


In [26]:
df

Car Name  \
0            BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1         MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2         Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+   
3                       BMW iX xDrive40BMW iX xDrive40   
4    Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...   
..                                                 ...   
968  Opel Vivaro-e Combi M 50 kWhOpel Vivaro-e Comb...   
969  Peugeot e-Expert Combi Long 50 kWhPeugeot e-Ex...   
970  Peugeot e-Traveller Compact 50 kWhPeugeot e-Tr...   
971  Citroën ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy...   
972  Mercedes-Benz EQE SUV 300 (MY23)Mercedes-Benz ...   

                                     Availability  Range (km)  \
0         Available to order since September 2025       610.0   
1           Available to order since October 2022       360.0   
2               Available to order since May 2025       585.0   
3      Discontinued (October 2021 - January 2025)       360.0   
4          Available to order since November 2020       235.0   
..                                            ...         ...   
968    Discontinued (October 2020 - January 2024)       185.0   
969   Discontinued (February 2021 - January 2024)       185.0   
970    Discontinued (January 2021 - January 2022)       185.0   
971       Discontinued (November 2020 - May 2022)       185.0   
972  Discontinued (September 2023 - October 2023)       440.0   

     Efficiency (Wh/km)  Weight (kg)  Acceleration (0-100s)  Battery (kWh)  \
0                 178.0       2360.0                    4.9          108.7   
1                 171.0       1720.0                    7.9           61.7   
2                 145.0       2050.0                    6.7           85.0   
3                 197.0       2440.0                    6.1           71.0   
4                 159.0       1360.0                    9.0           37.3   
..                  ...          ...                    ...            ...   
968               250.0       1960.0                   12.1           46.3   
969               250.0       1980.0                   13.1           46.3   
970               250.0       1930.0                   12.1           46.3   
971               250.0       1930.0                   12.1           46.3   
972               202.0       2460.0                    7.6           89.0   

     Fastcharge (kW)  Towing (kg)  Cargo (L)  Price (€)    Year Body Type  \
0              225.0       2000.0      578.0    68989.0  2025.0       SUV   
1              115.0        500.0      363.0    36589.0  2022.0     Other   
2              235.0       1500.0      506.0    55789.0  2025.0     Other   
3              102.0       2500.0      500.0    80889.0  2021.0       SUV   
4               67.0          0.0      185.0    33589.0  2020.0     Other   
..               ...          ...        ...        ...     ...       ...   
968             78.0       1000.0      507.0    47289.0  2020.0     Other   
969             78.0       1000.0      989.0    51589.0  2021.0     Other   
970             78.0       1000.0        NaN    50889.0  2021.0     Other   
971             78.0       1000.0      140.0    45489.0  2020.0     Other   
972            120.0        750.0      520.0    83789.0  2023.0       SUV   

     Page          Brand                                         Model Name  \
0       1            BMW              iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1       1             MG          MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2       1  Mercedes-Benz                     CLA 250+Mercedes-Benz CLA 250+   
3       1            BMW                         iX xDrive40BMW iX xDrive40   
4       1           Fiat    500e Hatchback 42 kWhFiat 500e Hatchback 42 kWh   
..    ...            ...                                                ...   
968     1           Opel  Vivaro-e Combi M 50 kWhOpel Vivaro-e Combi M 5...   
969     1        Peugeot  e-Expert Combi Long 50 kWhPeugeot e-Expert Com..

# 🧾 Step 9 — Convert dtypes & tidy column names 
**Why:** Ensure numeric columns are `float`/`Int64`, text is string, and column names are presentation-friendly for EDA.


In [28]:
# 🧾 Step 9 — Convert dtypes & tidy column names
# Purpose: ensure numeric columns are of proper numeric dtype, and clean up text whitespace

import pandas as pd
import numpy as np

numeric_cols_final = [
    "Real-World Range (km)", 
    "Energy Efficiency (Wh/km)", 
    "Battery Capacity (kWh)",
    "Vehicle Weight (kg)", 
    "Acceleration (0-100 km/h, sec)", 
    "Fast Charging Speed (kW)",
    "Towing Capacity (kg)", 
    "Cargo Volume (L)", 
    "Price (€)", 
    "Year",
    "Availability Start Year", 
    "Availability End Year"
]

# Convert to numeric safely (ignore if column missing)
for c in numeric_cols_final:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Convert year-like columns to nullable integer for better display
for c in ["Year", "Availability Start Year", "Availability End Year"]:
    if c in df.columns:
        df[c] = df[c].astype("Int64")

# Strip whitespace from text columns
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].str.strip()

print("✅ dtypes after conversion:")
print(df.dtypes)


✅ dtypes after conversion:
Car Name                    object
Availability                object
Range (km)                 float64
Efficiency (Wh/km)         float64
Weight (kg)                float64
Acceleration (0-100s)      float64
Battery (kWh)              float64
Fastcharge (kW)            float64
Towing (kg)                float64
Cargo (L)                  float64
Price (€)                  float64
Year                         Int64
Body Type                   object
Page                         int64
Brand                       object
Model Name                  object
Availability Start Year      Int64
Availability End Year        Int64
Vehicle Status              object
dtype: object


In [29]:
df

Car Name  \
0            BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1         MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2         Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+   
3                       BMW iX xDrive40BMW iX xDrive40   
4    Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...   
..                                                 ...   
968  Opel Vivaro-e Combi M 50 kWhOpel Vivaro-e Comb...   
969  Peugeot e-Expert Combi Long 50 kWhPeugeot e-Ex...   
970  Peugeot e-Traveller Compact 50 kWhPeugeot e-Tr...   
971  Citroën ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy...   
972  Mercedes-Benz EQE SUV 300 (MY23)Mercedes-Benz ...   

                                     Availability  Range (km)  \
0         Available to order since September 2025       610.0   
1           Available to order since October 2022       360.0   
2               Available to order since May 2025       585.0   
3      Discontinued (October 2021 - January 2025)       360.0   
4          Available to order since November 2020       235.0   
..                                            ...         ...   
968    Discontinued (October 2020 - January 2024)       185.0   
969   Discontinued (February 2021 - January 2024)       185.0   
970    Discontinued (January 2021 - January 2022)       185.0   
971       Discontinued (November 2020 - May 2022)       185.0   
972  Discontinued (September 2023 - October 2023)       440.0   

     Efficiency (Wh/km)  Weight (kg)  Acceleration (0-100s)  Battery (kWh)  \
0                 178.0       2360.0                    4.9          108.7   
1                 171.0       1720.0                    7.9           61.7   
2                 145.0       2050.0                    6.7           85.0   
3                 197.0       2440.0                    6.1           71.0   
4                 159.0       1360.0                    9.0           37.3   
..                  ...          ...                    ...            ...   
968               250.0       1960.0                   12.1           46.3   
969               250.0       1980.0                   13.1           46.3   
970               250.0       1930.0                   12.1           46.3   
971               250.0       1930.0                   12.1           46.3   
972               202.0       2460.0                    7.6           89.0   

     Fastcharge (kW)  Towing (kg)  Cargo (L)  Price (€)  Year Body Type  Page  \
0              225.0       2000.0      578.0    68989.0  2025       SUV     1   
1              115.0        500.0      363.0    36589.0  2022     Other     1   
2              235.0       1500.0      506.0    55789.0  2025     Other     1   
3              102.0       2500.0      500.0    80889.0  2021       SUV     1   
4               67.0          0.0      185.0    33589.0  2020     Other     1   
..               ...          ...        ...        ...   ...       ...   ...   
968             78.0       1000.0      507.0    47289.0  2020     Other     1   
969             78.0       1000.0      989.0    51589.0  2021     Other     1   
970             78.0       1000.0        NaN    50889.0  2021     Other     1   
971             78.0       1000.0      140.0    45489.0  2020     Other     1   
972            120.0        750.0      520.0    83789.0  2023       SUV     1   

             Brand                                         Model Name  \
0              BMW              iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1               MG          MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2    Mercedes-Benz                     CLA 250+Mercedes-Benz CLA 250+   
3              BMW                         iX xDrive40BMW iX xDrive40   
4             Fiat    500e Hatchback 42 kWhFiat 500e Hatchback 42 kWh   
..             ...                                                ...   
968           Opel  Vivaro-e Combi M 50 kWhOpel Vivaro-e Combi M 5...   
969        Peugeot  e-Expert Combi Long 50 kWhPeugeot e-Expert Com...   
9

In [30]:
df.dtypes

Car Name                    object
Availability                object
Range (km)                 float64
Efficiency (Wh/km)         float64
Weight (kg)                float64
Acceleration (0-100s)      float64
Battery (kWh)              float64
Fastcharge (kW)            float64
Towing (kg)                float64
Cargo (L)                  float64
Price (€)                  float64
Year                         Int64
Body Type                   object
Page                         int64
Brand                       object
Model Name                  object
Availability Start Year      Int64
Availability End Year        Int64
Vehicle Status              object
dtype: object

# 🧹 Step 10 — Final Cleanup: Drop Redundant Columns & Reorder Cleanly


In [31]:
# 🧹 Step 10 — Finalize DataFrame before saving
# Purpose: drop redundant/raw scrape columns and reorder the final dataset neatly

# Drop redundant or raw columns (only those that exist)
cols_to_drop = [
    "Car Name", "Body Type",
    "Availability", "Page"
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")

# Final column order you requested
final_columns = [
    "Brand", "Model Name",
    "Vehicle Status", "Year", "Availability Description",
    "Availability Start Year", "Availability End Year",
    "Range (km)", "Efficiency (Wh/km)","Battery (kWh)","Weight (kg)",
    "Acceleration (0-100s)", "Fastcharge (kW)",
    "Towing (kg)", "Cargo (L)", "Price (€)"
]

# Keep only columns that actually exist (avoids KeyError)
df = df[[c for c in final_columns if c in df.columns]]

# Final text cleanup
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].str.strip()

print("✅ Final DataFrame shape:", df.shape)
print("✅ Final columns:")
print(df.columns.tolist())

display(df.head(10))


✅ Final DataFrame shape: (973, 15)
✅ Final columns:
['Brand', 'Model Name', 'Vehicle Status', 'Year', 'Availability Start Year', 'Availability End Year', 'Range (km)', 'Efficiency (Wh/km)', 'Battery (kWh)', 'Weight (kg)', 'Acceleration (0-100s)', 'Fastcharge (kW)', 'Towing (kg)', 'Cargo (L)', 'Price (€)']


Brand                                         Model Name  \
0            BMW              iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1             MG          MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2  Mercedes-Benz                     CLA 250+Mercedes-Benz CLA 250+   
3            BMW                         iX xDrive40BMW iX xDrive40   
4           Fiat    500e Hatchback 42 kWhFiat 500e Hatchback 42 kWh   
5          CUPRA     Born 150 kW - 58 kWhCUPRA Born 150 kW - 58 kWh   
6          Tesla  Model 3 Long Range RWD (Highland)Tesla Model 3...   
7          Tesla  Model Y Long Range AWD (Juniper)Tesla Model Y ...   
8          CUPRA     Born 170 kW - 77 kWhCUPRA Born 170 kW - 77 kWh   
9            Kia                   EV3 Long RangeKia EV3 Long Range   

  Vehicle Status  Year  Availability Start Year  Availability End Year  \
0      Available  2025                     2025                   <NA>   
1      Available  2022                     2022                   <NA>   
2      Available  2025                     2025                   <NA>   
3   Discontinued  2021                     2021                   2025   
4      Available  2020                     2020                   <NA>   
5   Discontinued  2021                     2021                   2024   
6      Available  2025                     2025                   <NA>   
7   Discontinued  2025                     2025                   2025   
8   Discontinued  2022                     2022                   2024   
9      Available  2024                     2024                   <NA>   

   Range (km)  Efficiency (Wh/km)  Battery (kWh)  Weight (kg)  \
0       610.0               178.0          108.7       2360.0   
1       360.0               171.0           61.7       1720.0   
2       585.0               145.0           85.0       2050.0   
3       360.0               197.0           71.0       2440.0   
4       235.0               159.0           37.3       1360.0   
5       350.0               166.0           58.0       1810.0   
6       580.0               136.0           79.0       1820.0   
7       455.0               165.0           75.0       2070.0   
8       450.0               171.0           77.0       1940.0   
9       455.0               171.0           78.0       1880.0   

   Acceleration (0-100s)  Fastcharge (kW)  Towing (kg)  Cargo (L)  Price (€)  
0                    4.9            225.0       2000.0      578.0    68989.0  
1                    7.9            115.0        500.0      363.0    36589.0  
2                    6.7            235.0       1500.0      506.0    55789.0  
3                    6.1            102.0       2500.0      500.0    80889.0  
4                    9.0             67.0          0.0      185.0    33589.0  
5                    7.3             82.0          0.0      385.0    38589.0  
6                    5.2            120.0       1000.0      682.0    47889.0  
7                    4.8            124.0       1600.0      971.0    55589.0  
8                    7.0            115.0          0.0      385.0    46389.0  
9                    7.7            105.0       1000.0      485.0    41489.0

In [32]:
df

Brand                                         Model Name  \
0              BMW              iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1               MG          MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2    Mercedes-Benz                     CLA 250+Mercedes-Benz CLA 250+   
3              BMW                         iX xDrive40BMW iX xDrive40   
4             Fiat    500e Hatchback 42 kWhFiat 500e Hatchback 42 kWh   
..             ...                                                ...   
968           Opel  Vivaro-e Combi M 50 kWhOpel Vivaro-e Combi M 5...   
969        Peugeot  e-Expert Combi Long 50 kWhPeugeot e-Expert Com...   
970        Peugeot  e-Traveller Compact 50 kWhPeugeot e-Traveller ...   
971        Citroën  ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy Combi X...   
972  Mercedes-Benz        EQE SUV 300 (MY23)Mercedes-Benz EQE SUV 300   

    Vehicle Status  Year  Availability Start Year  Availability End Year  \
0        Available  2025                     2025                   <NA>   
1        Available  2022                     2022                   <NA>   
2        Available  2025                     2025                   <NA>   
3     Discontinued  2021                     2021                   2025   
4        Available  2020                     2020                   <NA>   
..             ...   ...                      ...                    ...   
968   Discontinued  2020                     2020                   2024   
969   Discontinued  2021                     2021                   2024   
970   Discontinued  2021                     2021                   2022   
971   Discontinued  2020                     2020                   2022   
972   Discontinued  2023                     2023                   2023   

     Range (km)  Efficiency (Wh/km)  Battery (kWh)  Weight (kg)  \
0         610.0               178.0          108.7       2360.0   
1         360.0               171.0           61.7       1720.0   
2         585.0               145.0           85.0       2050.0   
3         360.0               197.0           71.0       2440.0   
4         235.0               159.0           37.3       1360.0   
..          ...                 ...            ...          ...   
968       185.0               250.0           46.3       1960.0   
969       185.0               250.0           46.3       1980.0   
970       185.0               250.0           46.3       1930.0   
971       185.0               250.0           46.3       1930.0   
972       440.0               202.0           89.0       2460.0   

     Acceleration (0-100s)  Fastcharge (kW)  Towing (kg)  Cargo (L)  Price (€)  
0                      4.9            225.0       2000.0      578.0    68989.0  
1                      7.9            115.0        500.0      363.0    36589.0  
2                      6.7            235.0       1500.0      506.0    55789.0  
3                      6.1            102.0       2500.0      500.0    80889.0  
4                      9.0             67.0          0.0      185.0    33589.0  
..                     ...              ...          ...        ...        ...  
968                   12.1             78.0       1000.0      507.0    47289.0  
969                   13.1             78.0       1000.0      989.0    51589.0  
970                   12.1             78.0       1000.0        NaN    50889.0  
971                   12.1             78.0       1000.0      140.0    45489.0  
972                    7.6            120.0        750.0      520.0    83789.0  

[973 rows x 15 columns]

In [35]:
df.columns

Index(['Brand', 'Model Name', 'Vehicle Status', 'Year',
       'Availability Start Year', 'Availability End Year', 'Range (km)',
       'Efficiency (Wh/km)', 'Battery (kWh)', 'Weight (kg)',
       'Acceleration (0-100s)', 'Fastcharge (kW)', 'Towing (kg)', 'Cargo (L)',
       'Price (€)'],
      dtype='object')

### Rename columns to Meaningful Column Names in Data Frame

In [36]:
# Rename technical column names to more meaningful ones
rename_map = {
    "Range (km)": "Real-World Range (km)",
    "Efficiency (Wh/km)": "Energy Efficiency (Wh/km)",
    "Battery (kWh)": "Battery Capacity (kWh)",
    "Weight (kg)": "Vehicle Weight (kg)",
    "Acceleration (0-100s)": "Acceleration (0-100 km/h, sec)",
    "Fastcharge (kW)": "Fast Charging Speed (kW)",
    "Towing (kg)": "Towing Capacity (kg)",
    "Cargo (L)": "Cargo Volume (L)"
}

# Apply renaming only for columns that actually exist
df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns}, inplace=True)

print("✅ Column names renamed successfully.")
print("Updated columns:\n", df.columns.tolist())


✅ Column names renamed successfully.
Updated columns:
 ['Brand', 'Model Name', 'Vehicle Status', 'Year', 'Availability Start Year', 'Availability End Year', 'Real-World Range (km)', 'Energy Efficiency (Wh/km)', 'Battery Capacity (kWh)', 'Vehicle Weight (kg)', 'Acceleration (0-100 km/h, sec)', 'Fast Charging Speed (kW)', 'Towing Capacity (kg)', 'Cargo Volume (L)', 'Price (€)']


In [37]:
df

Brand                                         Model Name  \
0              BMW              iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1               MG          MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2    Mercedes-Benz                     CLA 250+Mercedes-Benz CLA 250+   
3              BMW                         iX xDrive40BMW iX xDrive40   
4             Fiat    500e Hatchback 42 kWhFiat 500e Hatchback 42 kWh   
..             ...                                                ...   
968           Opel  Vivaro-e Combi M 50 kWhOpel Vivaro-e Combi M 5...   
969        Peugeot  e-Expert Combi Long 50 kWhPeugeot e-Expert Com...   
970        Peugeot  e-Traveller Compact 50 kWhPeugeot e-Traveller ...   
971        Citroën  ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy Combi X...   
972  Mercedes-Benz        EQE SUV 300 (MY23)Mercedes-Benz EQE SUV 300   

    Vehicle Status  Year  Availability Start Year  Availability End Year  \
0        Available  2025                     2025                   <NA>   
1        Available  2022                     2022                   <NA>   
2        Available  2025                     2025                   <NA>   
3     Discontinued  2021                     2021                   2025   
4        Available  2020                     2020                   <NA>   
..             ...   ...                      ...                    ...   
968   Discontinued  2020                     2020                   2024   
969   Discontinued  2021                     2021                   2024   
970   Discontinued  2021                     2021                   2022   
971   Discontinued  2020                     2020                   2022   
972   Discontinued  2023                     2023                   2023   

     Real-World Range (km)  Energy Efficiency (Wh/km)  Battery Capacity (kWh)  \
0                    610.0                      178.0                   108.7   
1                    360.0                      171.0                    61.7   
2                    585.0                      145.0                    85.0   
3                    360.0                      197.0                    71.0   
4                    235.0                      159.0                    37.3   
..                     ...                        ...                     ...   
968                  185.0                      250.0                    46.3   
969                  185.0                      250.0                    46.3   
970                  185.0                      250.0                    46.3   
971                  185.0                      250.0                    46.3   
972                  440.0                      202.0                    89.0   

     Vehicle Weight (kg)  Acceleration (0-100 km/h, sec)  \
0                 2360.0                             4.9   
1                 1720.0                             7.9   
2                 2050.0                             6.7   
3                 2440.0                             6.1   
4                 1360.0                             9.0   
..                   ...                             ...   
968               1960.0                            12.1   
969               1980.0                            13.1   
970               1930.0                            12.1   
971               1930.0                            12.1   
972               2460.0                             7.6   

     Fast Charging Speed (kW)  Towing Capacity (kg)  Cargo Volume (L)  \
0                       225.0                2000.0             578.0   
1                       115.0                 500.0             363.0   
2                       235.0                1500.0             506.0   
3                       102.0                2500.0             500.0   
4                        67.0                   0.0             185.0   
..                        ...                   ...               ...   
968               

## 🚗 Final Dataset — Column Descriptions

| **Column Name**                    | **Description**                                                                                              |
| ---------------------------------- | --------------------------------------------------------------------------------------------------------------------------------- |
| **Brand**                          | The car manufacturer or brand name (e.g., BMW, Tesla, Kia).                                                                       |
| **Model Name**                     | The specific model and variant name of the vehicle. Sometimes includes battery or drive details (e.g., “Model 3 Long Range RWD”). |
| **Vehicle Status**                 | Indicates whether the vehicle is **Available**, **Upcoming**, or **Discontinued**.                                                |
| **Year**                           | The model year or main release year for the vehicle.                                                                              |
| **Availability Start Year**        | The year when the model became available for sale or order.                                                                       |
| **Availability End Year**          | The year when the model was discontinued (if applicable). Empty (NaN) means it’s still available.                                 |
| **Real-World Range (km)**          | The practical driving range of the car (in kilometers) based on real-world tests, not ideal lab results.                          |
| **Energy Efficiency (Wh/km)**      | How much energy the car uses per kilometer, in **watt-hours per kilometer**. Lower is better (more efficient).                    |
| **Battery Capacity (kWh)**         | The size of the battery pack, measured in **kilowatt-hours (kWh)** — higher capacity means longer range.                          |
| **Vehicle Weight (kg)**            | The total weight of the car (in kilograms).                                                                                       |
| **Acceleration (0-100 km/h, sec)** | How many seconds it takes the car to accelerate from **0 to 100 km/h** — lower values mean faster acceleration.                   |
| **Fast Charging Speed (kW)**       | The maximum rate at which the car can charge using a DC fast charger (in **kilowatts**).                                          |
| **Towing Capacity (kg)**           | The maximum weight the car can tow (in kilograms).                                                                                |
| **Cargo Volume (L)**               | The total available cargo or trunk space (in **liters**).                                                                         |
| **Price (€)**                      | The base or starting price of the vehicle, shown in **euros (€)**.                                                                |


# 🧾 Step 11 — Save Cleaned Dataset



In [38]:
output_path = r"C:\Users\kishore\OneDrive\Desktop\Web scrapping_REGEX\webscrapping_project\webscraping\EV_Data_Cleaned.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n✅ Cleaned dataset saved successfully at:\n{output_path}")


✅ Cleaned dataset saved successfully at:
C:\Users\kishore\OneDrive\Desktop\Web scrapping_REGEX\webscrapping_project\webscraping\EV_Data_Cleaned.csv


In [39]:
df

Brand                                         Model Name  \
0              BMW              iX3 50 xDrive (MY26)BMW iX3 50 xDrive   
1               MG          MG4 Electric 64 kWhMG MG4 Electric 64 kWh   
2    Mercedes-Benz                     CLA 250+Mercedes-Benz CLA 250+   
3              BMW                         iX xDrive40BMW iX xDrive40   
4             Fiat    500e Hatchback 42 kWhFiat 500e Hatchback 42 kWh   
..             ...                                                ...   
968           Opel  Vivaro-e Combi M 50 kWhOpel Vivaro-e Combi M 5...   
969        Peugeot  e-Expert Combi Long 50 kWhPeugeot e-Expert Com...   
970        Peugeot  e-Traveller Compact 50 kWhPeugeot e-Traveller ...   
971        Citroën  ë-Jumpy Combi XS 50 kWhCitroen e-Jumpy Combi X...   
972  Mercedes-Benz        EQE SUV 300 (MY23)Mercedes-Benz EQE SUV 300   

    Vehicle Status  Year  Availability Start Year  Availability End Year  \
0        Available  2025                     2025                   <NA>   
1        Available  2022                     2022                   <NA>   
2        Available  2025                     2025                   <NA>   
3     Discontinued  2021                     2021                   2025   
4        Available  2020                     2020                   <NA>   
..             ...   ...                      ...                    ...   
968   Discontinued  2020                     2020                   2024   
969   Discontinued  2021                     2021                   2024   
970   Discontinued  2021                     2021                   2022   
971   Discontinued  2020                     2020                   2022   
972   Discontinued  2023                     2023                   2023   

     Real-World Range (km)  Energy Efficiency (Wh/km)  Battery Capacity (kWh)  \
0                    610.0                      178.0                   108.7   
1                    360.0                      171.0                    61.7   
2                    585.0                      145.0                    85.0   
3                    360.0                      197.0                    71.0   
4                    235.0                      159.0                    37.3   
..                     ...                        ...                     ...   
968                  185.0                      250.0                    46.3   
969                  185.0                      250.0                    46.3   
970                  185.0                      250.0                    46.3   
971                  185.0                      250.0                    46.3   
972                  440.0                      202.0                    89.0   

     Vehicle Weight (kg)  Acceleration (0-100 km/h, sec)  \
0                 2360.0                             4.9   
1                 1720.0                             7.9   
2                 2050.0                             6.7   
3                 2440.0                             6.1   
4                 1360.0                             9.0   
..                   ...                             ...   
968               1960.0                            12.1   
969               1980.0                            13.1   
970               1930.0                            12.1   
971               1930.0                            12.1   
972               2460.0                             7.6   

     Fast Charging Speed (kW)  Towing Capacity (kg)  Cargo Volume (L)  \
0                       225.0                2000.0             578.0   
1                       115.0                 500.0             363.0   
2                       235.0                1500.0             506.0   
3                       102.0                2500.0             500.0   
4                        67.0                   0.0             185.0   
..                        ...                   ...               ...   
968               